<a href="https://colab.research.google.com/github/lucguittard/DS-Unit-2-Regression-Classification/blob/master/Luc_Guittard_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [X] Get the model's coefficients and intercept.
- [X] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [X] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [X] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [X] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [_] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

## Startup

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

## Train/Test Split 
Use data from April & May 2016 to train. Use data from June 2016 to test.

Run after setting new features.

In [0]:
#df.head()

In [68]:
df['description']=df['description'].astype(str)
df['description'].dtype

dtype('O')

In [69]:
df.dtypes #want created as manipulatable dtype
df['created'] = pd.to_datetime(df['created'], infer_datetime_format = True)
df['created'].dtype

dtype('<M8[ns]')

In [70]:
train_mask = (df['created'] > '2016-03-31') & (df['created'] < '2016-06-01')
train_df = df[train_mask]
train_df.head(20)
print(train_df.shape,df.shape)
# len(train_df.query('created == 2016-6')) how many 'created' for this date listed in train_df? -> 0

(31844, 37) (48817, 37)


In [0]:
test_mask = (df['created'] > '2016-05-31') & (df['created'] < '2016-07-01')
test_df = df[test_mask]
test_df.head()

## Adding Features
a) total no. rooms;

b) total no. perks;

c) description quality

In [43]:
print(df.shape)
df.head(2)

(48817, 36)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,perks
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5


In [0]:
#a)
df['rooms'] = df['bedrooms'] + df['bathrooms']

#b) 
df['perks'] = df.iloc[:, 10:34].sum(axis=1) 

In [0]:
#c) fxn
def string_eval(string):  #https://www.w3resource.com/python-exercises/python-functions-exercise-7.php
    description_quality={"UPPER_CASE":0, "LOWER_CASE":0}
    for c in string:
        if c.isupper():
           description_quality["UPPER_CASE"]+=1
        elif c.islower():
           description_quality["LOWER_CASE"]+=1
        else:
           pass
    ratio = description_quality['LOWER_CASE'] / (description_quality['UPPER_CASE'] + description_quality['LOWER_CASE'] + 0.000000001)
    print(ratio)
    return ratio

string_eval('A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ')

# c) application
df['description_quality'] = df['description'].apply(string_eval)

In [66]:
df.head(3)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,perks,description_quality
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,0.860731
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5,0.000000
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,3,0.966605


## Fit a Linear Regression Model

In [73]:
df.head(2)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,rooms,perks,description_quality
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0,0.860731
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,5,0.000000


In [75]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
features = ['rooms','perks']
target = 'price'
X = df[features]
y = df[target]

model.fit(X,y)
# model.intercept_,model.coef_
coefficients = pd.Series(model.coef_,features)

print("Intercept", model.intercept_)
print(coefficients.to_string())

Intercept 993.1828411176416
rooms    768.068774
perks    103.305690


## Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.

https://towardsdatascience.com/linear-regression-using-python-b136c91bf0a2

In [89]:
#training set

#Set-up
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

features = ['rooms','perks']
target = 'price'
X = train_df[features]
y = train_df[target]

#fitting
model = LinearRegression()
model.fit(X,y)

#prediction
y_pred = model.predict(X)

#evaluating
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y,y_pred))
r2 = r2_score(y,y_pred)

print(mae,rmse,r2)

#features = ['rooms','perks'] -> 856.6168448694106 1295.7149658677172 0.45930369872329946
#features = ['rooms','description_quality'] -> 893.4050430082522 1339.9182177175815 0.42178276518505176

856.6168448694106 1295.7149658677172 0.45930369872329946


In [87]:
#testing set

features = ['rooms','description_quality']
target = 'price'
X = test_df[features]
y = test_df[target]

#fitting
model = LinearRegression()
model.fit(X,y)

#prediction
y_pred = model.predict(X)

#evaluating
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y,y_pred))
r2 = r2_score(y,y_pred)

print(mae,rmse,r2)

#features = ['rooms','perks'] -> 869.8308311742767 1293.1031676824098 0.46189032953132003
#features = ['rooms','description_quality'] -> 910.7254150450506 1340.2734471979866 0.4219155690675813

910.7254150450506 1340.2734471979866 0.4219155690675813


Interpreting results: 

MAE - "the average magnitude of the errors in a set of predictions" (https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d).

Lower MAE suggests a a better predictive capacity of the model in question. So a model trained on the rooms and perks features to predict price is a more accurate one than a model trained on the rooms and description quality features to predict price.  